[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CCS-ZCU/pribehy-dat/blob/master/scripts/api.ipynb)


# API: Aplikační rozhraní

**autor**: *Vojtěch Kaše* (kase@ff.zcu.cz)

[![](https://ccs.zcu.cz/wp-content/uploads/2021/10/cropped-ccs-logo_black_space_240x240.png)](https://ccs.zcu.cz)

## Úvod a cíle kapitoly

Systém dotazování v podobě URL adres a odpovídání na ně v podobě dat, kterým jsme se zabývali v předchozí kapitole, je někdy standardizovaný do podoby tzv. **API**: Application Programming Interface, kdy vracená data již nemají podobu webových stránek, ale specificky strukturovaných dat, nad kterými vývojář zajásá. Tímto způsobem se na našich chytrých telefonech například aktualizují data o počasí (viz např. dokumentaci k API od [OpenWeatherMa](https://openweathermap.org/api/one-call-3#current)).  Aby však nedošlo k přetížení či zneužití těchto služeb, většina API vyžaduje nějakou formu autorizace, nejčastěji ve formě jakéhosi automaticky generovaného klíče či tokenu pro registrované uživatele.

Tato technologie je využívána i pro potřeby zpřístupňování dat z oblasti péče o kulturní dědictví. V této kapitole si ukážeme několik příkladů těchto webových služeb.

Projekt Epigraphic Database Heidelberg hostí digitalizovanou kolekci více než 80,000 převážně latinských nápisů z antického Říma. Tyto nápisy si veřejnost může prohlížet a prohledávat za využití webových stránek [zde](https://edh.ub.uni-heidelberg.de/inschrift/suche). Kromě toho má však badatel ještě jinou možnost, jak se dostat k příslušným datům, a totiž právě za využití speciálně vyvinutého API, které je zdokumentováno [zde](https://edh.ub.uni-heidelberg.de/data/api).

V této dokumentaci se podrobně dočteme, jaké parametry můžeme v našem dotazu (query) použít a jak. 

Základní URL adresa je `https://edh.ub.uni-heidelberg.de/data/api/inschrift/suche?`, za níž připojujeme parametry pro určení námi hledaného nápisu či skupiny nápisů.

Každý nápis v EDH databázi má svůj jednoznačný číslený identifikátor. Jako příklad může posloužit nápis s identifikačním číslem "HD000010", který je přes webové rozhraní přístupný [zde](https://edh.ub.uni-heidelberg.de/inschrift/suche?hd_nr=000010&tm_nr=&fo_antik=&fo_modern=&fundstelle=&region=&compFundjahr=eq&fundjahr=&aufbewahrung=&compHoehe=eq&hoehe=&compBreite=eq&breite=&compTiefe=eq&tiefe=&bh=&dat_tag=&dat_monat=&jahre=600+BC+-+1500+AD&literatur=&kommentar=&p_name=&p_praenomen=&p_nomen=&p_cognomen=&p_supernomen=&p_origo=&p_geschlecht=&p_ljahre_comp=gt&p_ljahre=&p_lmonate_comp=gt&p_lmonate=&p_ltage_comp=gt&p_ltage=&p_lstunden_comp=gt&p_lstunden=&atext1=&bool=AND&atext2=&sort=hd_nr&anzahl=20).

Známe-li ID nápisu, můžeme se dostat k datům o něm přes API takto:

```
https://edh.ub.uni-heidelberg.de/data/api/inschrift/suche?hd_nr=000010
```
Vyzkoušejme nejprve v prohlížeči! Vidíme datový obsah bez jakéhokoli formátování. 

Chceme-li např. získat data o všech nápisech z Římské provincie *Germania superior*, nastavíme parametr "provinz" na hodnotu "ges" (Tyto parametry a jejich hodnoty samozřejmě nestanovujeme z hlavy, ale snažíme se je vyčíst z dokumentace. Někdy však musíme trochu experimentovat). Výsledná URL adresa pak vypadá takto:

```
https://edh.ub.uni-heidelberg.de/data/api/inschrift/suche?provinz=ges
```

## Cvičení

In [ ]:
# naimportujeme si několik knihoven
import requests # python knihovna pro vznášení HTTP dotazů
from bs4 import BeautifulSoup # python knihovna pro práci s daty ve formátu html či xml
import pandas as pd

Nyní si to vyzkoušejme v praxi za využití Pythonu a knihoven requests, BeautifulSoup, a pandas, které jsme již použili v předchozích kapitolách.
* Naše výchozí adresa bude vypadat vždy stejně, uložme si ji do proměnné `base_url`.
* Následně specifikujme náš dotaz, tzv. `query`. 
* Tento dotaz nyní vzneseme prostřednictvím knihovny `requests` a odpověď si uložme jako objekt `resp`.

In [ ]:
base_url = 'https://edh.ub.uni-heidelberg.de/data/api/inschrift/suche?'
query = 'hd_nr=000010'
resp = requests.get(base_url + query)

Podíjme se na obsah atributu `.text`:

In [ ]:
resp.text

Vidíme totéž, co jsme viděli v prohlížeči. Na první pohled tato data možná nepůsobí příliš vábně, ale datový analytik zájásí. Použité speciální znaky jako "{", "}", "[", a "]" v kombinaci s dvojtečkami totiž indikují, že vrácená data jsou strukturována v tzv. **JSON** struktuře. JSON (=JavaScript Object Notation) je v současnosti velice populární způsob strukturovaného zápisu dat, s kterým si dokáže poradit velké množství programů. 

Pro uživatele Pythonu je JSON zvláště přitažlivý, neboť se jedná v podstatě o strukturu do sebe se zanořujících objektů typu `dict` a `list`. Takto tato data načteme do Pythonu prostřednictvím metody `json()`.

In [ ]:
data_json = resp.json()
data_json

Nyní již data vypadají úhledněji. Vzpomeňme na syntaxi objektů typu `dict` a `list`. Vypišme si všechny klíče našeho JSON objektu: 

In [ ]:
data_json.keys()

V tuto chvíli je pro nás nejzajímavější položka "items", kde jsou uschována data pro nápisy jako takové. Tyto "items" mají podobu datové struktury typu `list`. Jelikož však máme co do činění pouze s jedním nápisem, nachází se zde jediný prvek (viz též klíč "total"). 

K tomuto prvku se musíme dostat prostřednictvím indexování - jeho index bude 0.

Samotný prvek má opět podobu `dict`. Jednotlivé klíče pak definují jednotlivé proměnné definující daný nápis.
 

In [ ]:
data_json["items"][0]

Chceme-li získat text tohoto nápisu

In [ ]:
#Chceme-li získat text tohoto nápisu, použijeme klíč "transcription"
data_json["items"][0]["transcription"]

In [ ]:
#Chceme-li získat informaci o tom, ze kdy nápis pochází, použijeme klíče "not_before" a "not_after"
data_json["items"][0]["not_before"]

In [ ]:
# Tato buňka slouží ke kontrole průchodu tímto cvičením. 
# Pokud toto cvičení plníte v rámci svých studijních povinností na ZČU, buňku spusťte a držte se instrukcí.
exec(requests.get("https://sciencedata.dk/shared/856b0a7402aa7c7258186a8bdb329bd3?download").text)
kontrola_pruchodu(ntb="api", arg1=data_json)

Vznesme nyní dotaz na nápisy z území ČR (bude jich podle všeho pouze omezené množství, neboť na našem území Římané aktivně nepůsobili.

In [ ]:
base_url = 'https://edh.ub.uni-heidelberg.de/data/api/inschrift/suche?'
query = 'land=cz' ## &not_after=251'
resp = requests.get(base_url + query)
data_json = resp.json()
data_json

In [ ]:
# klíč total podává informaci o celkovém počtu vrácených položek:
data_json["total"]

Chceme-li získat text šestého nápisu v pořadí, budeme jej indexovat pomocí hodnoty 5 a následně použijeme klíč "transcription"

In [ ]:
data_json["items"][5]["transcription"]

V této podobě však práce s daty není zcela nejpraktičtější. Velká výhoda JSONu však je, že tato data můžeme snadno převést např. do tabulkové podoby objektu typu `pandas.DataFrame`. Klíče se automaticky stanou názvy sloupců: 

In [ ]:
data_df = pd.DataFrame(data_json["items"])
data_df

V případě, že náš dotaz odpovídá většímu množství položek, API nám často vrátí pouze první "stránku" hodnot. V případě EDH API je na první stránce pouze dvacet prvních hodnot. To je případ níže. 

In [ ]:
base_url = 'https://edh.ub.uni-heidelberg.de/data/api/inschrift/suche?'
query = 'provinz=ges' ## &not_after=251'
resp = requests.get(base_url + query)
data_json = resp.json()
data_json

In [ ]:
data_json["total"]

Nyní vidíme, že celkový počet položek je 6832 (viz `data_json["total"]`). Na stránce však máme pouze prvních dvacet. 
Chceme-li získat data z druhé stránky, musíme vznést nový dotaz nastavit v něm parametr "offset". 

In [ ]:
base_url = 'https://edh.ub.uni-heidelberg.de/data/api/inschrift/suche?'
query = 'provinz=ges&offset=20'
resp = requests.get(base_url + query)
data_json = resp.json()
data_json

Takto jsme získali dalších 20 položek. 

Abychom však získali všechny položky, musíme použít cyklus FOR. Budeme postupně po hodnotách dvacet zvyšovat hodnotu parametru offset a rozšiřovat náš list "all_items".

In [ ]:
# seznam offsets v celkové délce našich dat:
offsets = [n for n in range(0,data_json["total"], 20)]
offsets[:10]

In [ ]:
%%time
all_items = []
base_url = 'https://edh.ub.uni-heidelberg.de/data/api/inschrift/suche?' # zůstává stejné
for offset in offsets: # pro každý offset
    query = 'provinz=ges&offset=' + str(offset)
    resp = requests.get(base_url + query)
    data_json = resp.json()
    all_items.extend(data_json["items"])

In [ ]:
# Nyní máme data pro veškeré nápisy z provincie Germania superior
len(all_items)

In [ ]:
# data si převedeme do objektu typu dataframe
data_df = pd.DataFrame(all_items)
data_df.head(5)

In [ ]:
data_df.shape

Máme tak dataset o 6832 řádcích a 27 sloupcích.

In [ ]:
kontrola_pruchodu(ntb="api", arg1=query)

Každé API se chová trochu odlišně, ale základní logika, včetně pohybu po stránkách, bývá dosti podobná. Vždy je potřeba trochu experimentování.